<a href="https://colab.research.google.com/github/learningmatter-mit/uvvisml/blob/main/uvvisml_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

The code in this notebook uses Chemprop v1.x, which supports `python>=3.7, <3.9`. When the notebook was first published, Google Colab used a Python version that matched these requirements. As of July 2025, Google Colab uses Python 3.11, and it is challenging to make Colab run a version of Python other than its default, or to change the Colab Python kernel. As a result, this notebook includes several "hacks" to allow the code to run in Python 3.8. Most of the setup steps and the `%%py38` cell magic at the beginning of each cell can be removed if this notebook is executed locally rather than in Colab.

In [1]:
# Install Python 3.8 (the maximum version supported by Chemprop 1.x) - Colab uses Python 3.11 as of July 2025
# Copied from: https://raw.githubusercontent.com/j3soon/colab-python-version/refs/heads/main/scripts/py38.sh

!wget -O miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_23.11.0-2-Linux-x86_64.sh
!bash ./miniconda.sh -b -f -p /usr/local
!conda install -q -y jupyter google-colab traitlets=5.5.0 -c conda-forge  # should take ~2 minutes
!python -m ipykernel install --name "py38" --user
!rm ./miniconda.sh

--2025-07-30 14:38:23--  https://repo.anaconda.com/miniconda/Miniconda3-py38_23.11.0-2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131844786 (126M) [application/octet-stream]
Saving to: ‘miniconda.sh’

miniconda.sh        100%[===================>] 125.74M   130MB/s    in 1.0s    

2025-07-30 14:38:24 (130 MB/s) - ‘miniconda.sh’ saved [131844786/131844786]

PREFIX=/usr/local
Unpacking payload ...
                                                                                    
Installing base environment...





Preparing transaction: - \ | / - \ | / done
Executing transaction: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
installation finished.
    You currently have a

In [2]:
!python --version # Python 3.8.18

Python 3.8.18


In [3]:
!python3.8 -m pip install chemprop==1.7.1 numpy==1.24.4 # must specify numpy version > 1.22 to avoid C-API import error; should take ~3 minutes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 

In [4]:
# Start a persistent Python-3.8 REPL behind the scenes to run everything in for the below cell magic
# Otherwise (with the cell magic only), variables and package imports do not carry over from cell to cell

from IPython.core.magic import register_cell_magic
from jupyter_client import KernelManager
import json, atexit, textwrap

km = KernelManager(kernel_name="python3")      # path resolves to python3.8 if on $PATH
km.kernel_cmd = ["python3.8", "-m", "ipykernel_launcher", "-f", "{connection_file}"]
km.start_kernel()

kc = km.client()
kc.start_channels()

@atexit.register
def _clean():
    kc.stop_channels()
    km.shutdown_kernel(now=True)

# Create new cell magic for cells to run their code through a Python 3.8 interpreter instead of the Colab default Python

@register_cell_magic
def py38(line, cell):
    # send code, wait for reply, print results
    msg_id = kc.execute(textwrap.dedent(cell))
    while True:
        msg = kc.get_iopub_msg()
        if msg['parent_header'].get('msg_id') == msg_id:
            if msg['msg_type'] == 'stream':
                print(msg['content']['text'], end="")
            elif msg['msg_type'] == 'error':
                print("\n".join(msg['content']['traceback']))
            elif msg['msg_type'] == 'execute_result':
                print(msg['content']['data']['text/plain'])
            elif msg['msg_type'] == 'status' and msg['content']['execution_state'] == 'idle':
                break


/usr/local/lib/python3.11/dist-packages/jupyter_client/manager.py:132: UserWarning: Setting kernel_cmd is deprecated, use kernel_spec to start different kernels.
  warnings.warn("Setting kernel_cmd is deprecated, use kernel_spec to "


In [5]:
%%py38
import chemprop # sys.path.append('/usr/local/lib/python3.8/site-packages/')

In [6]:
!git clone https://github.com/learningmatter-mit/uvvisml

Cloning into 'uvvisml'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 147 (delta 11), reused 6 (delta 6), pack-reused 123 (from 1)
Receiving objects: 100% (147/147), 9.28 MiB | 7.17 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [7]:
%%py38
import pandas as pd
import os
os.chdir('uvvisml/uvvisml')

In [8]:
!cd uvvisml/uvvisml; bash get_model_files.sh # may take ~2-10 minutes (download speeds from Zenodo are typically ~1-5MB/s)

--2025-07-30 14:44:27--  https://zenodo.org/record/5573027/files/models.tar.gz
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.48.194, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/5573027/files/models.tar.gz [following]
--2025-07-30 14:44:28--  https://zenodo.org/records/5573027/files/models.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 651010218 (621M) [application/octet-stream]
Saving to: ‘models.tar.gz’

models.tar.gz       100%[===================>] 620.85M  2.68MB/s    in 8m 4s   

2025-07-30 14:52:32 (1.28 MB/s) - ‘models.tar.gz’ saved [651010218/651010218]



# Data

In [9]:
%%py38
test_file = 'data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv'
df = pd.read_csv(test_file)
df

                                                 smiles  ... peakwavs_max
0                   CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1  ...        376.0
1                   CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1  ...        392.0
2                   CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1  ...        396.0
3                   CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1  ...        400.0
4                   CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1  ...        413.0
...                                                 ...  ...          ...
1705           c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34  ...        424.0
1706           c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34  ...        432.0
1707  COc1cc(C)c(-c2cc(-c3c(C)cc(OC)cc3C)c3ccc4c(-c5...  ...        367.0
1708  N#Cc1c(N2CCCCC2)cc(-c2cccc3ccccc23)c2c1-c1cccc...  ...        358.0
1709        N#Cc1c(N2CCCC2)cc(-c2ccccc2)c2c1Cc1ccccc1-2  ...        382.0

[1710 rows x 3 columns]


# Make Predictions

## Predict experimental peak with model trained on combined training set

**Equivalent to command line:**

python uvvisml/predict.py --test_file uvvisml/data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv --property absorption_peak_nm_expt --method chemprop --preds_file test_preds.csv

In [10]:
%%py38
arguments = [
  '--test_path', test_file,
  '--preds_path', '/dev/null',
  '--checkpoint_dir', 'models/lambda_max_abs/chemprop/combined/production/fold_0',
  '--number_of_molecules', '2',
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

preds = [x[0] for x in preds]
df['peakwavs_max_pred'] = preds
df

Loading training args
/usr/local/lib/python3.8/site-packages/chemprop/utils.py:472: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vars(torch.load(path, map_location=lambda s

## Predict TDDFT peak in vacuum

**Equivalent to command line:**

python uvvisml/predict.py --test_file uvvisml/data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv --property vertical_excitation_eV_tddft --method chemprop --preds_file test_preds.csv

In [11]:
%%py38
arguments = [
  '--test_path', test_file,
  '--preds_path', '/dev/null',
  '--checkpoint_dir', 'models/lambda_max_abs_wb97xd3/chemprop/all_wb97xd3/production/fold_0',
  '--number_of_molecules', '1',
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

preds = [x[0] for x in preds] # predictions are in eV
df['peakwavs_max_pred'] = preds
df['peakwavs_max_pred'] = 1240/df['peakwavs_max_pred'] # convert from eV to nm
df

Loading training args
/usr/local/lib/python3.8/site-packages/chemprop/utils.py:472: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vars(torch.load(path, map_location=lambda s

## Predict experimental peak with model trained on Deep4Chem training set

**Equivalent to command line:**

python uvvisml/predict.py --test_file uvvisml/data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv --property absorption_peak_nm_expt --method chemprop --preds_file test_preds.csv --train_dataset deep4chem

In [12]:
%%py38
arguments = [
  '--test_path', test_file,
  '--preds_path', '/dev/null',
  '--checkpoint_dir', 'models/lambda_max_abs/chemprop/deep4chem/production/fold_0',
  '--number_of_molecules', '2',
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

preds = [x[0] for x in preds]
df['peakwavs_max_pred'] = preds
df

Loading training args
/usr/local/lib/python3.8/site-packages/chemprop/utils.py:472: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vars(torch.load(path, map_location=lambda s

## Predict experimental peak with multi-fidelity model

**Equivalent to command line:**

python uvvisml/predict.py --test_file uvvisml/data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv --property absorption_peak_nm_expt --method chemprop_tddft --preds_file test_preds.csv

In [13]:
%%py38
# TDDFT Predictions
arguments = [
  '--test_path', test_file,
  '--preds_path', 'test_tddft_preds.csv',
  '--checkpoint_dir', 'models/lambda_max_abs_wb97xd3/chemprop/all_wb97xd3/production/fold_0',
  '--number_of_molecules', '1',
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
_ = chemprop.train.make_predictions(args=args)

# Convert Predictions to Features File
!python models/tddft_to_features_file.py

# Experimental Predictions
arguments = [
  '--test_path', test_file,
  '--preds_path', '/dev/null',
  '--checkpoint_dir', 'models/lambda_max_abs/chemprop_tddft/combined/production/fold_0',
  '--number_of_molecules', '2',
  '--features_path', 'features_test.csv'
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

preds = [x[0] for x in preds]
df['peakwavs_max_pred'] = preds
df

Loading training args
/usr/local/lib/python3.8/site-packages/chemprop/utils.py:472: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vars(torch.load(path, map_location=lambda s

## Predict experimental peak with model trained on combined training set (with ensemble variance)

**Equivalent to command line:**

python uvvisml/predict.py --test_file uvvisml/data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv --property absorption_peak_nm_expt --method chemprop --preds_file test_preds.csv

In [14]:
%%py38
arguments = [
  '--test_path', test_file,
  '--preds_path', 'test_preds.csv',
  '--checkpoint_dir', 'models/lambda_max_abs/chemprop/combined/production/fold_0',
  '--number_of_molecules', '2',
  '--ensemble_variance',
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
_ = chemprop.train.make_predictions(args=args)

df = pd.read_csv('test_preds.csv')
df

Loading training args
/usr/local/lib/python3.8/site-packages/chemprop/utils.py:472: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vars(torch.load(path, map_location=lambda s